# Exploratory Data Analysis - Bot Detection Competition

This notebook performs initial exploratory data analysis (EDA) on the bot detection competition dataset to understand the characteristics of human vs. AI-generated essays.

## Objectives
1. Load and examine the structure of the training data
2. Analyze text length distributions
3. Examine word count statistics
4. Investigate punctuation usage patterns
5. Assess class balance (human vs. AI)
6. Generate initial hypotheses for feature engineering

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
import string
from collections import Counter
import textstat
from wordcloud import WordCloud

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("Libraries imported successfully!")

## 1. Data Loading and Initial Examination

In [ ]:
# Load the training data
train_essays = pd.read_csv('../data/train_essays.csv')
train_prompts = pd.read_csv('../data/train_prompts.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')

print("Dataset shapes:")
print(f"Training essays: {train_essays.shape}")
print(f"Training prompts: {train_prompts.shape}")
print(f"Sample submission: {sample_submission.shape}")

In [ ]:
# Examine the structure of training essays
print("Training Essays Info:")
print(train_essays.info())
print("\nFirst few rows:")
display(train_essays.head())

In [ ]:
# Examine training prompts
print("Training Prompts Info:")
print(train_prompts.info())
print("\nPrompt details:")
display(train_prompts[['prompt_id', 'prompt_name']].head())

In [ ]:
# Check for missing values
print("Missing values in training essays:")
print(train_essays.isnull().sum())
print("\nMissing values in training prompts:")
print(train_prompts.isnull().sum())

## 2. Class Balance Analysis

In [ ]:
# Analyze class distribution
class_counts = train_essays['generated'].value_counts()
class_proportions = train_essays['generated'].value_counts(normalize=True)

print("Class Distribution:")
print(f"Human-written (0): {class_counts[0]} ({class_proportions[0]:.2%})")
print(f"AI-generated (1): {class_counts[1]} ({class_proportions[1]:.2%})")

# Visualize class balance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Bar plot
class_counts.plot(kind='bar', ax=ax1, color=['skyblue', 'lightcoral'])
ax1.set_title('Class Distribution (Count)')
ax1.set_xlabel('Class (0=Human, 1=AI)')
ax1.set_ylabel('Count')
ax1.tick_params(axis='x', rotation=0)

# Pie chart
ax2.pie(class_counts.values, labels=['Human', 'AI'], autopct='%1.1f%%', 
        colors=['skyblue', 'lightcoral'])
ax2.set_title('Class Distribution (Proportion)')

plt.tight_layout()
plt.show()

## 3. Text Length Analysis

In [ ]:
# Calculate text lengths
train_essays['text_length'] = train_essays['text'].str.len()
train_essays['word_count'] = train_essays['text'].str.split().str.len()
train_essays['sentence_count'] = train_essays['text'].str.split('.').str.len() - 1

# Basic statistics for text length by class
text_stats = train_essays.groupby('generated')['text_length'].describe()
print("Text Length Statistics by Class:")
print(text_stats)

In [ ]:
# Visualize text length distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Distribution by class
for i, class_val in enumerate([0, 1]):
    class_name = 'Human' if class_val == 0 else 'AI'
    data = train_essays[train_essays['generated'] == class_val]['text_length']
    
    axes[0, i].hist(data, bins=50, alpha=0.7, color='skyblue' if class_val == 0 else 'lightcoral')
    axes[0, i].set_title(f'Text Length Distribution - {class_name}')
    axes[0, i].set_xlabel('Text Length (characters)')
    axes[0, i].set_ylabel('Frequency')

# Box plot comparison
train_essays.boxplot(column='text_length', by='generated', ax=axes[1, 0])
axes[1, 0].set_title('Text Length by Class (Box Plot)')
axes[1, 0].set_xlabel('Class (0=Human, 1=AI)')

# Violin plot
sns.violinplot(data=train_essays, x='generated', y='text_length', ax=axes[1, 1])
axes[1, 1].set_title('Text Length by Class (Violin Plot)')
axes[1, 1].set_xlabel('Class (0=Human, 1=AI)')

plt.tight_layout()
plt.show()

## 4. Word Count Analysis

In [ ]:
# Word count statistics by class
word_stats = train_essays.groupby('generated')['word_count'].describe()
print("Word Count Statistics by Class:")
print(word_stats)

# Visualize word count distributions
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
train_essays[train_essays['generated'] == 0]['word_count'].hist(bins=50, alpha=0.7, 
                                                                 label='Human', color='skyblue', ax=axes[0])
train_essays[train_essays['generated'] == 1]['word_count'].hist(bins=50, alpha=0.7, 
                                                                 label='AI', color='lightcoral', ax=axes[0])
axes[0].set_title('Word Count Distribution by Class')
axes[0].set_xlabel('Word Count')
axes[0].set_ylabel('Frequency')
axes[0].legend()

# Box plot
sns.boxplot(data=train_essays, x='generated', y='word_count', ax=axes[1])
axes[1].set_title('Word Count by Class')
axes[1].set_xlabel('Class (0=Human, 1=AI)')

plt.tight_layout()
plt.show()

## 5. Punctuation Usage Analysis

In [ ]:
def analyze_punctuation(text):
    """Analyze punctuation usage in text"""
    punctuation_counts = {}
    
    # Count specific punctuation marks
    punctuation_counts['periods'] = text.count('.')
    punctuation_counts['commas'] = text.count(',')
    punctuation_counts['exclamations'] = text.count('!')
    punctuation_counts['questions'] = text.count('?')
    punctuation_counts['semicolons'] = text.count(';')
    punctuation_counts['colons'] = text.count(':')
    punctuation_counts['quotations'] = text.count('"') + text.count("'")
    punctuation_counts['parentheses'] = text.count('(') + text.count(')')
    
    # Total punctuation
    total_punct = sum(1 for char in text if char in string.punctuation)
    punctuation_counts['total_punctuation'] = total_punct
    
    # Punctuation density (per 100 characters)
    text_len = len(text)
    punctuation_counts['punct_density'] = (total_punct / text_len * 100) if text_len > 0 else 0
    
    return punctuation_counts

# Apply punctuation analysis
punct_data = train_essays['text'].apply(analyze_punctuation)
punct_df = pd.DataFrame(list(punct_data))
punct_df['generated'] = train_essays['generated'].values

# Display punctuation statistics by class
print("Punctuation Statistics by Class:")
punct_stats = punct_df.groupby('generated')[['periods', 'commas', 'exclamations', 'questions', 
                                           'punct_density']].mean()
print(punct_stats)

In [ ]:
# Visualize punctuation usage
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Punctuation density
sns.boxplot(data=punct_df, x='generated', y='punct_density', ax=axes[0, 0])
axes[0, 0].set_title('Punctuation Density by Class')
axes[0, 0].set_xlabel('Class (0=Human, 1=AI)')

# Period usage
sns.boxplot(data=punct_df, x='generated', y='periods', ax=axes[0, 1])
axes[0, 1].set_title('Period Usage by Class')
axes[0, 1].set_xlabel('Class (0=Human, 1=AI)')

# Comma usage
sns.boxplot(data=punct_df, x='generated', y='commas', ax=axes[1, 0])
axes[1, 0].set_title('Comma Usage by Class')
axes[1, 0].set_xlabel('Class (0=Human, 1=AI)')

# Exclamation usage
sns.boxplot(data=punct_df, x='generated', y='exclamations', ax=axes[1, 1])
axes[1, 1].set_title('Exclamation Usage by Class')
axes[1, 1].set_xlabel('Class (0=Human, 1=AI)')

plt.tight_layout()
plt.show()

## 6. Readability Analysis

In [ ]:
def calculate_readability_scores(text):
    """Calculate various readability scores for text"""
    try:
        scores = {
            'flesch_kincaid': textstat.flesch_kincaid_grade(text),
            'flesch_reading_ease': textstat.flesch_reading_ease(text),
            'gunning_fog': textstat.gunning_fog(text),
            'coleman_liau': textstat.coleman_liau_index(text),
            'automated_readability': textstat.automated_readability_index(text),
            'avg_sentence_length': textstat.avg_sentence_length(text),
            'syllable_count': textstat.syllable_count(text),
        }
    except:
        # Handle cases where text might be empty or cause errors
        scores = {key: 0 for key in ['flesch_kincaid', 'flesch_reading_ease', 'gunning_fog', 
                                   'coleman_liau', 'automated_readability', 'avg_sentence_length', 
                                   'syllable_count']}
    return scores

# Calculate readability scores (sample first 1000 rows for faster computation)
sample_size = min(1000, len(train_essays))
sample_essays = train_essays.head(sample_size).copy()

print(f"Calculating readability scores for {sample_size} essays...")
readability_data = sample_essays['text'].apply(calculate_readability_scores)
readability_df = pd.DataFrame(list(readability_data))
readability_df['generated'] = sample_essays['generated'].values

# Display readability statistics
print("\nReadability Statistics by Class:")
readability_stats = readability_df.groupby('generated')[['flesch_kincaid', 'flesch_reading_ease', 
                                                        'gunning_fog', 'avg_sentence_length']].mean()
print(readability_stats)

In [ ]:
# Visualize readability scores
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Flesch-Kincaid Grade Level
sns.boxplot(data=readability_df, x='generated', y='flesch_kincaid', ax=axes[0, 0])
axes[0, 0].set_title('Flesch-Kincaid Grade Level by Class')
axes[0, 0].set_xlabel('Class (0=Human, 1=AI)')

# Flesch Reading Ease
sns.boxplot(data=readability_df, x='generated', y='flesch_reading_ease', ax=axes[0, 1])
axes[0, 1].set_title('Flesch Reading Ease by Class')
axes[0, 1].set_xlabel('Class (0=Human, 1=AI)')

# Gunning Fog Index
sns.boxplot(data=readability_df, x='generated', y='gunning_fog', ax=axes[1, 0])
axes[1, 0].set_title('Gunning Fog Index by Class')
axes[1, 0].set_xlabel('Class (0=Human, 1=AI)')

# Average Sentence Length
sns.boxplot(data=readability_df, x='generated', y='avg_sentence_length', ax=axes[1, 1])
axes[1, 1].set_title('Average Sentence Length by Class')
axes[1, 1].set_xlabel('Class (0=Human, 1=AI)')

plt.tight_layout()
plt.show()

## 7. Prompt Analysis

In [ ]:
# Analyze distribution across prompts
prompt_analysis = train_essays.groupby(['prompt_id', 'generated']).size().unstack(fill_value=0)
prompt_analysis['total'] = prompt_analysis.sum(axis=1)
prompt_analysis['ai_percentage'] = (prompt_analysis[1] / prompt_analysis['total'] * 100).round(2)

print("Essays per Prompt and Class:")
print(prompt_analysis)

# Merge with prompt names for better visualization
prompt_info = train_prompts[['prompt_id', 'prompt_name']]
prompt_analysis_named = prompt_analysis.reset_index().merge(prompt_info, on='prompt_id')

# Visualize prompt distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Stacked bar chart
prompt_analysis_named.set_index('prompt_name')[[0, 1]].plot(kind='bar', stacked=True, 
                                                           ax=axes[0], color=['skyblue', 'lightcoral'])
axes[0].set_title('Essays by Prompt and Class')
axes[0].set_xlabel('Prompt')
axes[0].set_ylabel('Number of Essays')
axes[0].legend(['Human', 'AI'])
axes[0].tick_params(axis='x', rotation=45)

# AI percentage by prompt
axes[1].bar(prompt_analysis_named['prompt_name'], prompt_analysis_named['ai_percentage'], 
           color='lightcoral')
axes[1].set_title('AI-Generated Essay Percentage by Prompt')
axes[1].set_xlabel('Prompt')
axes[1].set_ylabel('AI Percentage (%)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 8. Statistical Significance Testing

In [ ]:
from scipy import stats

# Perform statistical tests to determine if differences are significant
human_data = train_essays[train_essays['generated'] == 0]
ai_data = train_essays[train_essays['generated'] == 1]

# T-tests for continuous variables
tests = {
    'text_length': stats.ttest_ind(human_data['text_length'], ai_data['text_length']),
    'word_count': stats.ttest_ind(human_data['word_count'], ai_data['word_count']),
}

# Add punctuation tests
human_punct = punct_df[punct_df['generated'] == 0]
ai_punct = punct_df[punct_df['generated'] == 1]

punct_tests = {
    'punct_density': stats.ttest_ind(human_punct['punct_density'], ai_punct['punct_density']),
    'periods': stats.ttest_ind(human_punct['periods'], ai_punct['periods']),
    'commas': stats.ttest_ind(human_punct['commas'], ai_punct['commas']),
}

# Display results
print("Statistical Significance Tests (p < 0.05 indicates significant difference):")
print("\nText Features:")
for feature, (statistic, p_value) in tests.items():
    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
    print(f"{feature}: t={statistic:.3f}, p={p_value:.6f} {significance}")

print("\nPunctuation Features:")
for feature, (statistic, p_value) in punct_tests.items():
    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
    print(f"{feature}: t={statistic:.3f}, p={p_value:.6f} {significance}")

print("\nLegend: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")

## 9. Summary Statistics and Correlation Analysis

In [ ]:
# Create comprehensive feature dataset
features_df = train_essays[['id', 'prompt_id', 'generated', 'text_length', 'word_count', 'sentence_count']].copy()
features_df = features_df.merge(punct_df[['periods', 'commas', 'exclamations', 'questions', 
                                        'punct_density', 'generated']], on='generated')

# Calculate correlation matrix
feature_cols = ['text_length', 'word_count', 'sentence_count', 'periods', 'commas', 
               'exclamations', 'questions', 'punct_density']
correlation_matrix = features_df[feature_cols + ['generated']].corr()

# Visualize correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
           square=True, fmt='.3f')
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Show correlations with target variable
target_correlations = correlation_matrix['generated'].drop('generated').abs().sort_values(ascending=False)
print("\nFeatures ranked by absolute correlation with target (generated):")
for feature, corr in target_correlations.items():
    print(f"{feature}: {corr:.4f}")

## 10. Key Findings and Initial Hypotheses

In [ ]:
# Summary of key findings
print("=== KEY FINDINGS FROM EXPLORATORY DATA ANALYSIS ===")
print()

print("1. CLASS BALANCE:")
print(f"   - Human essays: {class_counts[0]} ({class_proportions[0]:.2%})")
print(f"   - AI essays: {class_counts[1]} ({class_proportions[1]:.2%})")
print(f"   - Dataset is heavily imbalanced towards human essays")
print()

print("2. TEXT LENGTH PATTERNS:")
human_avg_length = train_essays[train_essays['generated'] == 0]['text_length'].mean()
ai_avg_length = train_essays[train_essays['generated'] == 1]['text_length'].mean()
print(f"   - Human essays average length: {human_avg_length:.0f} characters")
print(f"   - AI essays average length: {ai_avg_length:.0f} characters")
print(f"   - Difference: {abs(human_avg_length - ai_avg_length):.0f} characters")
print()

print("3. WORD COUNT PATTERNS:")
human_avg_words = train_essays[train_essays['generated'] == 0]['word_count'].mean()
ai_avg_words = train_essays[train_essays['generated'] == 1]['word_count'].mean()
print(f"   - Human essays average word count: {human_avg_words:.0f} words")
print(f"   - AI essays average word count: {ai_avg_words:.0f} words")
print(f"   - Difference: {abs(human_avg_words - ai_avg_words):.0f} words")
print()

print("4. PUNCTUATION USAGE:")
human_punct_density = punct_df[punct_df['generated'] == 0]['punct_density'].mean()
ai_punct_density = punct_df[punct_df['generated'] == 1]['punct_density'].mean()
print(f"   - Human essays punctuation density: {human_punct_density:.2f}%")
print(f"   - AI essays punctuation density: {ai_punct_density:.2f}%")
print(f"   - Difference: {abs(human_punct_density - ai_punct_density):.2f}%")
print()

print("5. MOST DISCRIMINATIVE FEATURES:")
for i, (feature, corr) in enumerate(target_correlations.head(5).items(), 1):
    print(f"   {i}. {feature}: {corr:.4f}")
print()

print("=== INITIAL HYPOTHESES FOR MODEL DEVELOPMENT ===")
print()
print("1. FEATURE ENGINEERING OPPORTUNITIES:")
print("   - Text length and word count appear to be discriminative")
print("   - Punctuation patterns may help distinguish AI from human text")
print("   - Readability scores could capture complexity differences")
print("   - N-gram analysis might reveal linguistic patterns")
print()
print("2. MODELING CONSIDERATIONS:")
print("   - Class imbalance needs to be addressed (sampling or class weights)")
print("   - Prompt-specific features might be valuable")
print("   - Ensemble methods could combine different feature types")
print("   - Cross-validation should stratify by both class and prompt")
print()
print("3. DATA AUGMENTATION POSSIBILITIES:")
print("   - Generate more AI essays using different LLMs")
print("   - Vary prompt engineering techniques")
print("   - Consider synthetic minority oversampling techniques")

print("\n=== NEXT STEPS ===")
print("1. Implement advanced feature engineering pipeline")
print("2. Address class imbalance")
print("3. Develop baseline models")
print("4. Set up experiment tracking with MLflow")